In [1]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

In [ ]:
llm.invoke("Write an essay on Elon Musk")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

In [ ]:
chain = prompt | llm 

In [ ]:
chain.invoke({"input": "Write an essay on Elon Musk"})

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [ ]:
chain = prompt | llm | output_parser

In [ ]:
chain.invoke({"input": "Write an essay on Elon Musk"})

In [2]:
from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="llama3")

In [3]:
from PyPDF2 import PdfReader
pdf_docs="documents\Budget_Speech_small.pdf"
pdf_reader= PdfReader(pdf_docs)

In [4]:
text=""
for page in pdf_reader.pages:
    text+= page.extract_text()

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
chunks = text_splitter.split_text(text)

In [6]:
from langchain_community.vectorstores import FAISS
vector_store = FAISS.from_texts(chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

In [11]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

retriever = vector_store.as_retriever()

retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Hon’ble Speaker, I present the Budget for 2023 -24. This is the first Budget in Amrit Kaal. Introduction \n1. This Budget hopes to build on the foundation laid in the previous Budget, and the blueprint drawn \nfor India@100. We envision a prosperous and inclusive India, in which the fruits of development \nreach all regions and citizens, especially our youth, women, farmers, OBCs, Scheduled Castes and \nScheduled Tribes. 2. In the 75t h year of our Independence, the world has recognised the Indian \neconomy as a ‘bright star’. Our current year’s economic growth is estimated to be at 7 per cent. It is \nnotable that this is the highest among all the major economies. This is in spite of the m assive \nslowdown globally caused by Covid -19 and a war. The Indian economy is therefore on the right \ntrack, and despite a time of challenges, heading towards a bright future. 3. Today as Indians stands \nwith their head held high, and the world appreciates In dia’s achiev